<a href="https://colab.research.google.com/github/joeywangyolo/Starsky_Canvas/blob/main/%E6%A9%9F%E7%A5%A8LineBOT%EF%BC%BF%E5%AF%A6%E5%81%9A%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError,
    LineBotApiError
)
from linebot.models import *

In [ ]:
app = Flask(__name__,static_url_path='/material',static_folder='./material')
run_with_ngrok(app)

line_bot_api = LineBotApi('LineBotApi')
handler = WebhookHandler("WebhookHandler")

In [ ]:
from linebot.http_client import HttpResponse
#LineBot 的主程序對話入口
#啟動server對外接口，使Line能丟消息進來
@app.route("/" , methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text = True)
    print(body)


    try:
        events=handler.handle(body, signature)
        #handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
flex_message1 = FlexSendMessage(
    alt_text='hello',
    contents=BubbleContainer(
        direction='ltr',
        hero=ImageComponent(
            url='https://www.e-japannavi.com/syuyu/images/map_cts_hokkaido_img_hakodate.jpg',
            size='full',
            aspect_ratio='20:13',
            aspect_mode='cover',
            action=URIAction(uri='http://example.com', label='label')
        )
    )
)

In [ ]:
import json
from linebot.models import (
    TextSendMessage,ImageSendMessage,FlexSendMessage,PostbackEvent
)
with open('/content/1階段_1.json',encoding='utf8') as f:
  flight_message = json.load(f)
  FlexSendMessage.new_from_json_dict(flight_message)
print(flight_message)
  

In [ ]:
import re
@handler.add(MessageEvent,message = TextMessage)
#當用戶傳送文字消息時的處理
def handle_message(event):

  with open('/content/1階段_1.json',encoding='utf8') as f: 
    flight_message = json.load(f)

  if event.message.text.find('@')!=-1 :
    line_bot_api.reply_message(
        event.reply_token,
        #template_message_dict.get(event.message.text),
        FlexSendMessage.new_from_json_dict(flight_message)
    )
    #data 為用戶輸入的日期變數，之後用 date 來做爬蟲的時間參數 
  elif re.match('[2022.2023.2024.2025]+-\d{1,2}-\d{1,2}',event.message.text):
    date =event.message.text
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='您要選擇的日期為：'+date),
        #TextSendMessage(text='請輸入 @日本機票 來查看日本的重點城市')
    )
  else:
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='字典內查無此字，請輸入@日本機票')
    )

In [ ]:
from linebot.models import PostbackEvent

@handler.add(PostbackEvent)
def handle_post_message(event):
    print(event.postback.data)
    #postback data 找到 HKD(用戶點了HKD之後)
    if (event.postback.data.find('HKD')== 0):
      #這邊呼叫爬蟲def (date , 涵館)
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，將為您找尋前往 涵館 的相關資訊',
                    #FlexSendMessage(第二階段的範本訊息)
                )
            )
    elif (event.postback.data.find('TYOA') == 0):
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，將為您找尋前往 東京 的相關資訊',
                )
            )
    elif (event.postback.data.find('OSAA') == 0):
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，將為您找尋前往 大阪 的相關資訊'
                )
            )
    elif (event.postback.data.find('KOJ') == 0):
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，將為您找尋前往 鹿兒島 的相關資訊'
                )
            )
    elif (event.postback.data.find('OKA') == 0):
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，將為您找尋前往 沖繩 的相關資訊'
                )
            )                    
    else:
        pass

In [ ]:
app.run()